Pour reproduire ce notebook, installer:
* requests
* dotenv
* pydub -> échantillonage de l'audio
* audioop-lts -> idem

In [1]:
import requests
import json
from dotenv import load_dotenv
import os

load_dotenv("/home/chougar/Documents/GitHub/experiments/.env")

API_KEY_REF=os.getenv("OPENROUTER_API_KEY")


url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {API_KEY_REF}",
    "Content-Type": "application/json"
}




In [ ]:
# fonction pour obtenir un échantillon du audio du fichier source pour le tester, si celui ci est trop long
def get_audio_sample(input_path: str, fract_sample=0.2)-> str:
    """
        Inputs:
            * input_path: chemin complet vers l'audio (mp3 requis) 
            * fract_sample: % de l'échantillon à produire (0.2 par défaut)
        Output:
            * chemin complet de l'échantillon généré
    """
    if ".mp3" not in input_path:
        return ".mp3 requis"
    
    from pydub import AudioSegment

    output_path= f"{input_path.split(".mp3")[0]}_sample.mp3"
    print(f"Echantillon dans {output_path}")    

    # 1. Chargement du fichier
    try:
        # Pydub peut charger directement depuis un MP3
        audio = AudioSegment.from_mp3(input_path)
    except Exception as e:
        print(f"Erreur lors du chargement ou ffmpeg non trouvé : {e}")
        exit()

    # 2. Calcul de la durée
    # Pydub stocke la durée totale en millisecondes
    total_duration_ms = len(audio) 

    # Affichage informatif
    print(f"Durée totale de l'audio : {total_duration_ms / 1000:.2f} secondes")

    # 3. Calcul du point de coupe (Le premier tiers)
    # Nous voulons : [0ms : 1/3 de la durée totale]
    duree_premier_tiers_ms = total_duration_ms * fract_sample    

    # Conversion en entier pour le slicing
    end_time_ms = int(duree_premier_tiers_ms)

    print(f"Extraction du segment [0s : {end_time_ms / 1000:.2f}s]")


    # 4. Découpage (Slicing) de l'audio
    # Pydub utilise la notation de tranche Python : [début_ms : fin_mes]
    premier_tiers = audio[0:end_time_ms]

    # 5. Exportation du nouveau segment
    # Le format est spécifié dans l'argument 'format'
    premier_tiers.export(output_path, format="mp3")

    print(f"Fichier exporté avec succès à : {output_path}")    

    return output_path

# fonction pour obtenir un échantillon du audio du fichier source pour le tester, si celui ci est trop long
def get_audio_chunks(input_path: str, user_max_duration_min=15) -> list:
    """
        Objectif:
            * dans le cas des audio dépassant la limite du LLM (15 minutes pour gemini flash), décomposer le fichier
            en n fragments de 15 minutes max
        Inputs:
            * input_path: chemin complet vers l'audio (mp3 requis)             
        Output:
            * liste des chemins vers les fragments audio
    """
    if ".mp3" not in input_path:
        return ".mp3 requis"
    
    from pydub import AudioSegment

    output_path_template= f"{input_path.split(".mp3")[0]}"    

    # 1. Chargement du fichier
    try:
        # Pydub peut charger directement depuis un MP3
        audio = AudioSegment.from_mp3(input_path)
    except Exception as e:
        print(f"Erreur lors du chargement ou ffmpeg non trouvé : {e}")
        exit()

    # 2. Calcul de la durée
    # Pydub stocke la durée totale en millisecondes
    total_duration_ms = len(audio) 

    # Affichage informatif
    print(f"Durée totale de l'audio : {total_duration_ms / 1000:.2f} secondes")

    # fragmenter si durée totale > 15min
    def build_time_intervals(max_duration, total_duration_ms):
        import math          

        nb_chunks=math.ceil(1/(max_duration/total_duration_ms))

        start_time, end_time=(0, max_duration)
        time_intervals=[]
        for i in range(1, nb_chunks+1):
            time_intervals.append([start_time, start_time+end_time])
            start_time+=end_time

        print(time_intervals)
        last_time_computed=time_intervals[-1][0]
        last_time_effective= last_time_computed+(total_duration_ms-last_time_computed)

        # update last time interval
        time_intervals[-1][1]=last_time_effective

        return time_intervals

    max_duration=user_max_duration_min*60*1000 #60 seconds, 1000ms in 1s
    if (total_duration_ms)>max_duration:
        time_intervals=build_time_intervals(max_duration, total_duration_ms)

        audio_chunks_paths=[]
        p=1
        for t in time_intervals[:1]:
            print(f"Extraction du segment [{t[0]} : {t[1]}]")
            # Découpage (Slicing) de l'audio
            chunk=audio[t[0]:t[1]]
            # 5. Exportation du nouveau segment
            # Le format est spécifié dans l'argument 'format'
            output_path=f"{output_path_template}_p{p}.mp3"
            chunk.export(output_path, format="mp3")

            print(f"Fichier exporté avec succès à : {output_path}")    
            
            audio_chunks_paths.append(output_path)

            p+=1
    

    return audio_chunks_paths


# fonction pour transformer le format audio source (mp3) en format accepté par le llm
def encode_audio_to_base64(audio_path):
    import base64
    with open(audio_path, "rb") as audio_file:
        return base64.b64encode(audio_file.read()).decode('utf-8')


In [ ]:
input_path = "/home/chougar/Téléchargements/L-IA-notre-deuxieme-conscience.mp3"
op=get_audio_chunks(input_path)
op

Durée totale de l'audio : 2539.91 secondes


In [3]:
# Chemin vers votre fichier d'entrée
input_path = "/home/chougar/Téléchargements/journal.mp3"
audio_path= input_path

# active ce code pour obtenir un échantillon de l'audio (20% par défaut)
audio_path=get_audio_sample(input_path, fract_sample=0.3)


# Read and encode the audio file
base64_audio = encode_audio_to_base64(audio_path)

Echantillon dans /home/chougar/Téléchargements/journal_sample.mp3
Durée totale de l'audio : 464.17 secondes
Extraction du segment [0s : 139.25s]
Fichier exporté avec succès à : /home/chougar/Téléchargements/journal_sample.mp3


Appel au llm

In [4]:
diarization_prompt_v2 = """
    Veuillez transcrire ce fichier audio, qui est un extrait d'émission de radio avec plusieurs locuteurs.
    1. Transcrivez l'intégralité du dialogue.
    2. Identifiez chaque locuteur séquenciellement (Locuteur 1, Locuteur 2, etc.).
    3. Utilisez le format '[Locuteur X]: [Texte du locuteur]'.
    4. Séparez chaque intervention par des sauts de ligne standard (`\n` ou `<br>`).

    APRES la transcription complète, vous devez fournir un seul bloc de code JSON avec la liste des associations entre l'ID du locuteur et son nom réel (ou une description si le nom est inconnu, comme 'narrateur radio').

    Ce JSON doit impérativement utiliser des guillemets doubles standard (") et être encadré par des triples accents ````json et ````.

    Exemple de format de sortie final :

    [Locuteur 1]: Bienvenue sur Radio X. Et voici le journal du jour ...
    [Locuteur 2]: Bonjour Marot. Bonjour Toto, bonne nuit à toutes et à tous.
    ...
    [Locuteur X]: Le ministre de la débrouille veut exploser...

    ```json
    {
        "Locuteur 1": "Toto",
        "Locuteur 2": "Marot",
        "Locuteur 3": "Davido" 
    }
"""

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": diarization_prompt_v2
            },
            {
                "type": "input_audio",
                "input_audio": {
                    "data": base64_audio,
                    "format": "mp3"
                }
            }
        ]
    }
]

payload = {
    "model": "google/gemini-2.5-flash",
    "messages": messages
}

response = requests.post(url, headers=headers, json=payload)



In [5]:
raw_output=response.json()["choices"][0]["message"]["content"]
print(raw_output)

[Locuteur 1]: France Culture.
[Locuteur 2]: Et voici donc le premier journal, celui de Margot Delpierre. Bonjour Margot.
[Locuteur 3]: Bonjour Guillaume. Bonjour à toutes et à tous. Le ministre de l'économie veut rassurer un an avant la réforme des découverts bancaires, les règles pour ceux de moins de 200 € et sur de courtes périodes seront alignées sur celles des crédits à la consommation. L'Union européenne appelle les autorités à la retenue en Tanzanie après plusieurs jours de violences électorales, il y aurait des centaines de morts.
[Locuteur 2]: Cette semaine, la série Grand reportage est consacrée aux attentats de Paris et de Saint-Denis, 10 ans après le premier épisode juste après ce journal.
[Locuteur 3]: Sera-t-il possible de tenir les délais ? Aujourd'hui se termine à l'Assemblée l'examen du volet recette du budget de l'État pour 2026, mais le vote initialement prévu demain n'aura pas lieu. Il reste plus de 2000 amendements à étudier. Place finalement au débat sur le PLFSS,

In [6]:
# fonction pour extraire les locuteurs en json
def extract_json_locuteurs_from_audio_output(text: str) -> json:
    json_start="```json"
    if json_start in raw_output:
        json_end=raw_output.rfind("}")
        if json_end==-1:
            return "Error: JSON end not detected ('}')"

        dict_locuteurs=text[text.find(json_start)+len(json_start): text.rfind("}")+1]    
        try:
            dict_locuteurs= json.loads(dict_locuteurs)
            return dict_locuteurs
        except Exception as e:
            return f"Error: JSON conversion impossible: {e}"

    else:
        return f"Error: JSON start not detected ({json_start})"

In [7]:
dict_locuteurs= extract_json_locuteurs_from_audio_output(raw_output)
dict_locuteurs

{'Locuteur 1': 'Jingle Radio France Culture',
 'Locuteur 2': 'Guillaume Erner (Présentateur radio)',
 'Locuteur 3': 'Margot Delpierre (Journaliste)',
 'Locuteur 4': 'Analyste politique / Reporter'}

In [10]:
# fonction pour remplacer les id locteurs par les noms réels
def apply_locuteurs_names(dict_locuteurs: json, raw_output: str) -> str:
    final=raw_output
    for k,v in dict_locuteurs.items():
        final=final.replace(k, v)
    return final.split("```json")[0]

In [11]:
final_audio_to_text=apply_locuteurs_names(dict_locuteurs, raw_output)
print(final_audio_to_text)

[Jingle Radio France Culture]: France Culture.
[Guillaume Erner (Présentateur radio)]: Et voici donc le premier journal, celui de Margot Delpierre. Bonjour Margot.
[Margot Delpierre (Journaliste)]: Bonjour Guillaume. Bonjour à toutes et à tous. Le ministre de l'économie veut rassurer un an avant la réforme des découverts bancaires, les règles pour ceux de moins de 200 € et sur de courtes périodes seront alignées sur celles des crédits à la consommation. L'Union européenne appelle les autorités à la retenue en Tanzanie après plusieurs jours de violences électorales, il y aurait des centaines de morts.
[Guillaume Erner (Présentateur radio)]: Cette semaine, la série Grand reportage est consacrée aux attentats de Paris et de Saint-Denis, 10 ans après le premier épisode juste après ce journal.
[Margot Delpierre (Journaliste)]: Sera-t-il possible de tenir les délais ? Aujourd'hui se termine à l'Assemblée l'examen du volet recette du budget de l'État pour 2026, mais le vote initialement prévu